## First we initialize our globals

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from databricks.connect import DatabricksSession
spark = DatabricksSession.builder.getOrCreate()

In [4]:
# add project dir to python path
try:
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    import sys, os
    sys.path.append("/Workspace" + os.path.dirname(os.path.dirname(notebook_path)))
except:
    pass

# Read in cow Data

In [8]:
# load table into spark dataframe
cows_bff = spark.read.table("field_demos.db.cows_bff")

## Some graphs and charts

## Lets compute meal overlap

In [9]:
from cow_bff.heatmap import compute_heatmap

df = compute_heatmap(cows_bff)

display(df.limit(10))

,cow1,cow2,distance
0,Brandy,Buttercup,1232.923077
1,Brandy,Butterscotch,1162.461538
2,Brandy,Cocoa,1000.307692
3,Brandy,Daisy,819.384615
4,Brandy,Dottie,1130.538462
5,Brandy,Hershey,1146.230769
6,Brandy,Magic,1489.076923
7,Brandy,Marshmallow,1152.538462
8,Brandy,Mocha,1000.307692
9,Brandy,Muffin,1016.307692


# Display Heatmap

In [10]:
pdf = df.toPandas()
pdf = pdf.pivot(index='cow1', columns='cow2', values='distance').fillna(0)

import plotly.express as px
fig = px.imshow(pdf, x=pdf.columns, y=pdf.index, labels=dict(x="Cow 2", y="Cow 1", color="Distance"), title="Cow BFFs", color_continuous_scale='redor')
px.imshow(pdf, x=pdf.columns, y=pdf.index, labels=dict(x="Cow 2", y="Cow 1", color="Distance"),)

fig.update_layout(width=800,height=500)
fig.show()
